# Personalize 排名示例<a class="anchor" id="top"></a>

在此笔记本中，您将选择一个数据集，以准备将该数据集用于 Amazon Personalize 批量推荐。

1. [选择数据集或数据源](#source)
1. [准备数据](#prepare)
1. [创建数据集组和交互数据集](#group_dataset)
1. [配置 S3 存储桶和 IAM 角色](#bucket_role)
1. [导入交互数据](#import)
1. [创建解决方案](#solutions)
1. [创建活动](#create)
1. [与活动交互](#interact)
1. [清理](#cleanup)

## 简介<a class="anchor" id="intro"></a>

在大部分情况下，Amazon Personalize 中的算法（称为配方）在很大程度上寻求解决不同的任务，如下所述：

1. **HRNN 和 HRNN-Metadata** – 根据以前用户与项的交互推荐项目。
1. **HRNN-Coldstart** – 推荐尚未提供交互数据的新项目。
1. **Personalized-Ranking** – 获取一个项目集合，然后使用类似 HRNN 的方法按照可能感兴趣的顺序对它们进行排序。
1. **SIMS（类似项目）**– 根据给定的项目，推荐用户也可交互的其他项目。
1. **Popularity-Count** – 如果 HRNN 或 HRNN-Metadata 没有答案，则推荐最受欢迎的项目 – 这是默认返回值。

无论使用案例如何，所有算法都共享基于用户-项目-交互数据的学习基础，该数据由 3 个核心属性定义：

1. **UserID** – 参与交互的用户
1. **ItemID** – 用户与之交互的项目
1. **Timestamp** – 交互发生的时间

我们还支持按以下值定义的事件类型和事件值：

1. **事件类型** – 事件的分类标签（浏览、已购买、已评价等）。
1. **事件值** – 与发生的事件类型对应的值。一般来说，我们在事件类型中寻找 0 和 1 之间的归一化值。例如，如果完成交易有三个阶段（点击、添加到购物车和购买），则每个阶段的事件值分别为 0.33、0.66 和 1.0。

事件类型和事件值字段是附加数据，可用于筛选为训练个性化模型而发送的数据。在这个特定的练习中，我们没有事件类型或事件值。

## 选择数据集或数据源<a class="anchor" id="source"></a>
[返回页首](#top)

正如我们所提到的，用户-项目-交互数据是开始使用服务的关键。这意味着我们需要寻找生成这种数据的使用案例，几个常见示例包括：

1. 视频点播应用
1. 电子商务平台
1. 社交媒体聚合器/平台

使用一些指导原则可帮助确定适合 Personalize 的问题的范围。我们建议将下面的值作为起点，不过[官方限制](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)略低一些。

* 经过身份验证的用户
* 至少 50 个独立用户
* 至少 100 个独立项目
* 每个用户至少 24 次交互 

大多数情况下，这是很容易实现的，如果您在一个类别中排名较低，您通常可以通过在另一个类别中拥有更高排名来弥补。

一般来说，您的数据不会是适合 Personalize 的完美形式，需要进行一些修改才能正确结构化。此笔记本会指导您完成所有这些任务。

首先，我们将使用 [Last.FM](https://grouplens.org/datasets/hetrec-2011/) 数据集。这些数据是有关用户的音乐收听行为的记录。这些数据符合我们的指导原则，包括大量的用户、项目和交互。

首先，您将下载数据集并使用下面的代码将数据集解压缩到一个新文件夹中。

In [4]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

--2020-05-26 20:45:51--  http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  10.2MB/s    in 0.2s    

2020-05-26 20:45:51 (10.2 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


看看您下载的数据文件。

In [5]:
!ls $data_dir

artists.dat		  tags.dat	    user_taggedartists.dat
hetrec2011-lastfm-2k.zip  user_artists.dat  user_taggedartists-timestamps.dat
readme.txt		  user_friends.dat


目前，除了我们似乎掌握的许多 .dat 文件和 README 外，我们对这些数据还知之甚少。打开 README 将告诉我们有关该数据的整体结构。除非数据源来自外部团队，否则对于自定义数据，您可以跳过这个步骤。

从 README 中，我们可以看到此数据集中有多个交互类型。将彼此标记为朋友的用户之间的交互、收听艺术家的用户的交互以及分配给用户和艺术家的标签的交互。

在这种情况下，我们关注的是用户、艺术家和收听交互。我们有 1892 个用户，17632 个艺术家（在这种情况下是我们的项目），以及 92834 个用户-收听艺术家交互。这足以让我们开始使用 Personalize。

继续阅读 README，进入 `Files` 部分。数据集中的大多数文件与我们无关，但 `users_artists.dat` 文件看起来很有希望。README 的 `Data format` 部分提供了有关该文件内容的更多详细信息。这是我们遇到第一个问题的地方。

| userID | artistID | 权重  |
|--------|----------|---------|
| 2      | 51       | 13883   |

尽管在用户和他们正在收听的艺术家之间存在交互数据，但是这些交互被存储为权重而不是时间戳。在 Amazon Personalize 中，我们需要的是用户-项目-时间戳交互数据。

如果您再次查看数据集中的文件，您应该看到 `users_taggedartists-timestamps.dat` 确实包含时间戳数据。那么，如果我们使用标记行为作为我们的交互数据，而不是收听行为呢？ 我们可以假设用户标记艺术家是一种积极情绪的表现吗？ 通常，您会与您的客户或具有领域知识的人进行讨论，以了解此交互是否适合您想要解决的用例。现在，我们将假设标记行为适合我们的需要。

`user_taggedartists-timestamps.dat` 的架构为：

| userID | artistID | tagID | 时间戳     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

如果我们删除 `tagID` 属性，我们就会得到 Amazon Personalize 所需的格式。

## 准备数据<a class="anchor" id="prepare"></a>
[返回页首](#top)

接下来要做的是加载数据并确认数据处于良好状态，然后将数据保存到 CSV 中，以便用于 Amazon Personalize。

首先，导入数据科学中常用的 Python 库集合。

In [6]:
import time
from time import sleep
import json
from datetime import datetime
import numpy as np
import boto3
import pandas as pd

接下来，打开数据文件并查看前几行。

In [7]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

,userID artistID tagID timestamp
0,2\t52\t13\t1238536800000
1,2\t52\t15\t1238536800000
2,2\t52\t18\t1238536800000
3,2\t52\t21\t1238536800000
4,2\t52\t41\t1238536800000


很明显，数据没有正确加载。CSV（逗号分隔值）文件的默认分隔符是逗号（`,`），但在这种情况下，文件是使用制表符（`\t`）分隔符保存的。所以，让我们指定正确的分隔符并再次尝试加载数据。

In [8]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


这样就好多了。现在数据已经成功加载到内存中，让我们提取一些额外的信息。首先，从数据中计算出一些基本的统计数据。

In [9]:
original_data.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


这表明我们有一个很好的 `userID` 和 `artistID` 值范围。接下来，确认数据格式始终是一个好主意。

In [10]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186479 entries, 0 to 186478
Data columns (total 4 columns):
userID       186479 non-null int64
artistID     186479 non-null int64
tagID        186479 non-null int64
timestamp    186479 non-null int64
dtypes: int64(4)
memory usage: 5.7 MB


由此可以看出，数据集中总共有 186479 个条目，有 4 列，每个单元格都存储为 int64 格式。

Int64 格式显然适用于 `userID` 和 `artistID`。但是，我们需要更深入地了解数据中的时间戳。要使用 Amazon Personalize，您需要以 [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) 格式保存时间戳。

目前，时间戳值不是人类可读的。因此，让我们获取一个任意的时间戳值，并找出如何解释它。

In [11]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000000


ValueError: year 41132 is out of range

哎呀！对于这个特定的时间戳值，代码呈现的年份为 41,132。这对我们来说有点遥远，所以很明显这不是解析数据的正确方法。我们需要第二次尝试。

JavaScript 以毫秒为单位记录时间，这是来自 Web 应用程序的数据集合，因此在应用代码之前，让我们将时间戳值除以 1000。

In [12]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009-02-28 23:00:00


对于我们的数据集来说，2009 年 2 月感觉更加真实。虽然我们不需要人类可读的时间戳也可使用 Amazon Personalize，但我们确实希望日期是真实的，所以现在我们将数据集中的每个时间戳从 JavaScript 毫秒格式转换过来，以继续下面的操作。 

In [13]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1.238537e+09
1,2,52,15,1.238537e+09
2,2,52,18,1.238537e+09
3,2,52,21,1.238537e+09
4,2,52,41,1.238537e+09


通过选择任意时间戳并将时间戳转换为人类可读的格式，对转换后的数据集进行快速的可用性检查。

In [14]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000.0
2009-02-28 23:00:00


这个日期作为时间戳是有意义的，因此我们可以继续格式化其余的数据。请记住，我们需要的数据是用户-项目-交互数据，在本例中是 `userID`、`artistID` 和 `timestamp`。我们的数据集有一个额外的列，即 `tagID`，可将它从数据集中删除。

In [15]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

,userID,artistID,timestamp
0,2,52,1.238537e+09
1,2,52,1.238537e+09
2,2,52,1.238537e+09
3,2,52,1.238537e+09
4,2,52,1.238537e+09


操作数据后，请务必确认数据格式是否已更改。

In [16]:
interactions_df.dtypes

userID         int64
artistID       int64
timestamp    float64
dtype: object

在本例中，时间戳列已从 int64 更改为 float64。因此，让我们将格式改回 int64。

In [17]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

userID       int64
artistID     int64
timestamp    int64
dtype: object

 Amazon Personalize 为用户、项目和时间戳提供了默认列名。这些默认列名为 `USER_ID`、`ITEM_ID` 和 `TIMESTAMP`。因此，对数据集的最后修改是用默认标题替换现有的列标题。

In [18]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

就是这样！此时，数据已准备就绪，我们只需将数据保存为 CSV 文件。

In [19]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## 创建数据集组和交互数据集<a class="anchor" id="group_dataset"></a>
[返回页首](#top)

Amazon Personalize 的最高隔离和抽象级别是*数据集组*。存储在其中一个数据集组中的信息不会影响任何其他数据集组或从其中一个数据集组创建的模型，这些数据集组是完全隔离的。这允许您运行许多实验，是我们如何保持您的模型的私密性并仅对您的数据进行全面训练的一部分。

在导入先前准备好的数据之前，需要有一个数据集组和一个添加到其中的数据集来处理交互。

数据集组可以包含以下类型的信息：

* 用户-项目-交互
* 事件流（实时交互）
* 用户元数据
* 项目元数据

在为交互数据创建数据集组和数据集之前，让我们验证您的环境是否可以成功地与 Amazon Personalize 通信。

In [20]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 创建数据集组

下面的单元格将创建一个名为 `personalize-poc-lastfm` 的新数据集组。

In [21]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-ranking-dsg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/personalize-ranking-dsg",
  "ResponseMetadata": {
    "RequestId": "386224ac-8410-4f64-ba4a-460ba12607c7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:46:27 GMT",
      "x-amzn-requestid": "386224ac-8410-4f64-ba4a-460ba12607c7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


在使用数据集组之前，它必须处于活跃状态。这可能需要一两分钟。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次数据集组的状态，工作最长可达 3 小时。

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


现在，您有了一个数据集组，您可以为交互数据创建一个数据集。

### 创建数据集

首先，定义一个架构，告诉 Amazon Personalize 您正在上载的数据集的类型。根据数据集的类型，架构中需要几个保留关键字和强制关键字。更详细的信息可以在[文档](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)中找到。

在这里，您将为需要 `USER_ID`、`ITEM_ID` 和 `TIMESTAMP` 字段的项目元数据创建架构。它们在架构中的定义顺序必须与它们在数据集中的出现顺序相同。

In [23]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ranking-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/personalize-ranking-interactions",
  "ResponseMetadata": {
    "RequestId": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:47:58 GMT",
      "x-amzn-requestid": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


创建架构后，可以在数据集组中创建数据集。请注意，这还不会加载数据。加载数据将在几个步骤后发生。

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ranking-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/personalize-ranking-dsg/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "98a3c225-31aa-49be-936f-f633aa6314d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:48:02 GMT",
      "x-amzn-requestid": "98a3c225-31aa-49be-936f-f633aa6314d0",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## 配置 S3 存储桶和 IAM 角色<a class="anchor" id="bucket_role"></a>
[返回页首](#top)

到目前为止，我们已下载、操作数据并将数据保存到运行此 Jupyter 笔记本的实例所连接的 Amazon EBS 实例中。但是，Amazon Personalize 需要一个 S3 存储桶充当您的数据源以及用于访问该存储桶的 IAM 角色。让我们把一切都安排好。

使用存储在此 Amazon SageMaker 笔记本底层的实例上的元数据来确定运行区域。如果您使用的是 Amazon SageMaker 以外的 Jupyter 笔记本，只需将区域定义为下面的字符串。Amazon S3 存储桶需要与我们目前创建的 Amazon Personalize 资源位于同一区域。

In [25]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 存储桶名称是全局唯一的。要创建唯一的存储桶名称，下面的代码会将字符串 `personalizepoc` 附加到您的 AWS 账号。然后，它会使用此名称在前一个单元格中发现的区域创建一个存储桶。

In [26]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-ranking-demo-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-ranking-demo-41600


### 将数据上载到 S3

现在，您的 Amazon S3 存储桶已经创建，请上载包含我们的用户-项目-交互数据的 CSV 文件。 

In [27]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### 设置 S3 存储桶策略
Amazon Personalize 需要能够读取 S3 存储桶的内容。因此，添加可实现此操作的存储桶策略。

In [28]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '61279AD0FB422CD2',
  'HostId': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
   'x-amz-request-id': '61279AD0FB422CD2',
   'date': 'Tue, 26 May 2020 20:48:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### 创建 IAM 角色

Amazon Personalize 需要能够承担 AWS 中的角色，以便拥有执行某些任务的权限。让我们来创建一个 IAM 角色并将所需的策略附加到该角色。下面的代码附加了非常宽松的策略；请对任何生产应用程序使用更严格的策略。

In [29]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRanking"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleRanking


## 导入交互数据<a class="anchor" id="import"></a>
[返回页首](#top)

之前，您创建了数据集组和数据集来存放您的信息，因此现在您将执行一个导入作业，将数据从 S3 存储桶加载到 Amazon Personalize 数据集。 

In [30]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ranking-example",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/personalize-ranking-example",
  "ResponseMetadata": {
    "RequestId": "7eab862d-9724-41dc-87d3-7455b4f16ead",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:51:57 GMT",
      "x-amzn-requestid": "7eab862d-9724-41dc-87d3-7455b4f16ead",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


在使用数据集之前，导入作业必须处于活跃状态。执行下面的单元格并等待单元格显示 ACTIVE 状态。它每秒检查一次导入作业的状态，工作最长可达 3 小时。

导入数据可能需要一些时间，具体取决于数据集的大小。在此研讨会中，数据导入作业大约需要 15 分钟。

In [31]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 50 ms, sys: 15.1 ms, total: 65 ms
Wall time: 13min


当数据集导入处于活跃状态时，您可以开始使用 SIMS、Personalized-Ranking、Popularity-Count 和 HRNN 构建模型。此过程将在其他笔记本中继续。运行下面的单元格，然后继续存储一些值，以便在下一个笔记本中使用。

## 创建解决方案<a class="anchor" id="solutions"></a>
[返回页首](#top)

在本笔记本中，您将使用以下配方创建解决方案：

1. HRNN


在 Amazon Personalize 中，算法的特定变体称为配方。不同的配方适用于不同的情况。经过训练的模型称为解决方案，每个解决方案可以有许多版本，这些版本与模型训练时给定的数据量相关。

首先，我们将列出支持的所有配方。这将允许您选择一个配方并使用它来构建模型。

In [32]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

### 个性化排名

个性化排名是 HRNN 的一个有趣应用。该算法不仅为相关用户推荐最有可能的内容，还接受用户和项目列表。然后按照可能与用户最相关的顺序重新呈现这些项目。这里的用例用于筛选流派，例如，当您有一个广泛的集合，您希望更好地为特定用户排序时。

对于我们的用例，请使用 LastFM 数据，我们可以想象一家唱片公司正在付钱给我们，以便在特别促销中向我们的用户推荐他们的艺术家。因此，我们知道要推荐的艺术家列表，但我们想知道每个用户最喜欢这些艺术家中的哪一位。我们将根据每个用户之前的标记历史，使用个性化排名为每个用户重新排序艺术家列表。

就像上次一样，我们从选择配方开始。

In [33]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### 创建解决方案

与之前的解决方案一样，首先创建解决方案。尽管您在此步骤中提供了数据集 ARN，但模型尚未经过训练。将模型视为标识符，而不是经过训练的模型。

In [34]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-ranking",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking",
  "ResponseMetadata": {
    "RequestId": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 创建解决方案版本

拥有解决方案之后，您需要创建一个版本来完成模型训练。使用我们的数据集，完成训练可能需要一段时间，最多 25 分钟，平均 35 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，这里的目标是创建许多模型并快速部署它们。因此，我们将为笔记本中的所有解决方案设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [35]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [36]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819",
  "ResponseMetadata": {
    "RequestId": "b07c347c-48aa-4650-86f6-53197351a62f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "b07c347c-48aa-4650-86f6-53197351a62f",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 查看解决方案创建状态

如前面所述，如何在控制台中查看状态更新呢：

* 在另一个浏览器选项卡中，您应该已经在打开此笔记本实例时启动了 AWS 控制台。
* 切换到该选项卡并在顶部搜索服务 `Personalize`，然后转到该服务页面。
* 单击 `View dataset groups`。
* 单击数据集组的名称，名称中很可能包含 POC。
* 单击 `Solutions and recipes`。
* 现在，您将看到您在上面创建的所有解决方案的列表，包括一个包含解决方案版本状态的列。在状态变成 `Active` 之后，您的解决方案就可以接受审核了。它也能够被部署。

也可简单地运行下面的单元格以跟踪解决方案版本的创建状态。

In [38]:
in_progress_solution_versions = [
    rerank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819


## 创建活动<a class="anchor" id="create"></a>
[返回页首](#top)

活动是托管解决方案版本；可以查询推荐的端点。通过估计吞吐量（用户每秒的个性化请求）来设置定价。部署活动时，您可以设置每秒最小吞吐量（TPS）值。与 AWS 中的许多服务一样，此服务将根据需求自动扩展，但如果延迟至关重要，您可能希望提前预置以满足更大的需求。对于此 POC 和演示，所有最小吞吐量阈值均设置为 1。有关更多信息，请参阅[定价页面](https://aws.amazon.com/personalize/pricing/)。

让我们开始部署活动。

### 个性化排名

为您的个性化排名解决方案版本部署活动。部署一个活动大约需要 10 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，并且这里的目标是创建多个活动。因此，我们将在笔记本中进一步为所有活动设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [39]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:48:55 GMT",
      "x-amzn-requestid": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 查看活动创建状态

如前面所述，如何在控制台中查看状态更新呢：

* 在另一个浏览器选项卡中，您应该已经在打开此笔记本实例时启动了 AWS 控制台。
* 切换到该选项卡并在顶部搜索服务 `Personalize`，然后转到该服务页面。
* 单击 `View dataset groups`。
* 单击数据集组的名称，名称中很可能包含 POC。
* 单击 `Campaigns`。
* 现在，您将看到上面创建的所有活动的列表，包括一个活动状态列。处于 `Active` 状态后，就可以查询您的活动了。

或者，只需运行下面的单元格即可跟踪活动创建状态。

In [41]:
in_progress_campaigns = [
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank


## 与活动交互<a class="anchor" id="interact"></a>
[返回页首](#top)

现在，所有活动都已部署并处于活跃状态，我们可以开始通过 API 调用获取推荐。每个活动都基于不同的配方，活动的行为方式略有不同，因为它们服务于不同的用例。我们将以不同于以前笔记本中使用的顺序介绍每个活动，以便按升序处理可能的复杂活动（即先处理最简单的活动）。

首先，让我们创建一个支持函数来帮助理解 Personalize 活动返回的结果。Personalize 仅返回一个 `item_id`。这对于保持数据紧凑非常有用，但这意味着您需要查询数据库或查找表，才能获得笔记本的用户可读结果。我们将创建一个 helper 函数，用于从 LastFM 数据集返回用户可读的结果。

首先加载数据集，我们可以将数据集用于我们的查找表。

In [42]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


通过将 ID 列定义为索引列，只需查询 ID 即可返回艺术家。

In [43]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


这并不可怕，但在我们的代码中到处重复这一点会变得很混乱，所以下面的函数将对查询进行清理。

In [44]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

现在，让我们测试几个简单的值来检查我们的错误捕获。

In [45]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


棒极了！现在我们有了一种呈现结果的方法。 

In [47]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


### 个性化排名

个性化排名的核心用例是收集项目，并按照用户兴趣的优先顺序或可能的顺序呈现项目。为了演示这一点，我们将需要一个随机用户和一个包含 25 个项目的随机集合。

In [48]:
rerank_user = users_df.sample(1).index.tolist()[0]
rerank_items = items_df.sample(25).index.tolist()

现在，构建一个显示输入数据的良好数据框。

In [49]:
rerank_list = []
for item in rerank_items:
    artist = get_artist_by_id(item)
    rerank_list.append(artist)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,1303
0,Priestess
1,Пелагея
2,Cosmic Gate
3,Mujuice
4,Prism
5,WIZO
6,Love of Lesbian
7,Mayhem
8,Taking Dawn
9,Universum


然后，进行个性化排名 API 调用。

In [50]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 26 May 2020 22:02:55 GMT',
   'x-amzn-requestid': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
   'content-length': '1442',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '1047', 'score': 0.1730128},
  {'itemId': '1046', 'score': 0.1639514},
  {'itemId': '1254', 'score': 0.1101557},
  {'itemId': '18692', 'score': 0.0762716},
  {'itemId': '5783', 'score': 0.0678702},
  {'itemId': '13434', 'score': 0.067667},
  {'itemId': '8915', 'score': 0.0616803},
  {'itemId': '1305', 'score': 0.0322442},
  {'itemId': '8543', 'score': 0.0297265},
  {'itemId': '11979', 'score': 0.0286533},
  {'itemId': '6820', 'score': 0.025196},
  {'itemId': '4154', 'score': 0.0246835},
  {'itemId': '2312', 'score': 0.0228279},
  {'itemId': '2158', 'score': 0.0179717},
  {'itemId': '2036', 'score': 0.0176177},
 

现在，将重新排序的项目作为第二列添加到原始数据框中，以便进行并排比较。

In [51]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    ranked_list.append(artist)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,1303,Re-Ranked
0,Priestess,McFly
1,Пелагея,New Found Glory
2,Cosmic Gate,Mayhem
3,Mujuice,Farid Farjad
4,Prism,Priestess
5,WIZO,Пелагея
6,Love of Lesbian,How to Destroy Angels
7,Mayhem,Cosmic Gate
8,Taking Dawn,Prism
9,Universum,Murder by Death


您可以在上面看到每个条目是如何根据模型对用户的理解重新排序的。当您有一组要向用户展示的项目（例如促销列表），或者您正在筛选一个类别并希望显示可能最好的项目时，这是一项常见任务。

## 清理活动

从清理活动开始。您需要先删除活动，然后才能删除活动所基于的解决方案版本。

下面的代码将列出您账户中的所有活动。

In [ ]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

查看 ARN 列表以确定要删除哪些活动。然后使用以下代码通过插入 ARN 来删除活动。

In [ ]:
personalize.delete_campaign(
    campaignArn = "INSERT ARN HERE"
)

## 清理解决方案

接下来，清理解决方案。下面的代码将列出您账户中的所有解决方案。

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

查看 ARN 列表以确定要删除的解决方案。然后使用下面的代码通过插入 ARN 来删除解决方案。

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## 清理数据集

接下来，清理数据集。下面的代码将列出您账户中的所有数据集。

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

查看 ARN 列表以确定要删除的数据集。然后使用下面的代码通过插入 ARN 来删除解决方案。

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## 清理架构

接下来，清理架构。下面的代码将列出您账户中的所有架构。

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

查看 ARN 列表以确定要删除的架构。然后使用下面的代码通过插入 ARN 来删除解决方案。

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## 清理数据集组

最后，清理数据集组。下面的代码将列出您账户中的所有数据集组。

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

查看 ARN 列表以确定要删除的数据集组。然后使用下面的代码通过插入 ARN 来删除解决方案。

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## 清除 S3 存储桶和 IAM 角色

（可选）您可以删除我们在研讨会中创建的 IAM 角色和 S3 存储桶。

首先使用下面的代码列出您账户中的所有 IAM 角色。

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

确定要删除的角色的名称。

不能删除仍附加有策略的 IAM 角色。因此，在您确定了相关角色之后，让我们列出该角色的附加策略。

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

您需要使用下面的代码分离上面结果中的策略。对附加的每个策略重复此操作。

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

最后，您应该能够删除 IAM 角色。

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

要删除 S3 存储桶，首先它需要是空的。删除 S3 存储桶最简单的方法是在 AWS 控制台中导航到 S3，删除存储桶中的对象，然后删除 S3 存储桶本身。